In [ ]:
# |default_exp tasks
# |default_cls_lvl 3

In [ ]:
# |hide
%reload_ext autoreload
%autoreload 2

# tasks

Code for running experiments and hyper parameters optimization from the terminal

In [ ]:
# |export
from __future__ import annotations

import argparse, os, warnings
from pathlib import Path

from dotenv import load_dotenv
from transformers import logging as hf_logging

from course_copilot import utils, training, topic_segmentation, summarization

In [ ]:
# | hide
import pdb

from fastcore.test import *
import nbdev

from blurr.utils import print_versions

In [ ]:
# |export
# silence all the HF warnings and load environment variables
warnings.simplefilter("ignore")
hf_logging.set_verbosity_error()

os.environ["TOKENIZERS_PARALLELISM"] = "false"
load_dotenv()

try:
    from nbdev.imports import IN_NOTEBOOK
except:
    IN_NOTEBOOK = False

In [ ]:
# | echo: false
print("What we're running with at the time this documentation was generated:")
print_versions("torch fastai transformers")

What we're running with at the time this documentation was generated:
torch: 1.12.1+cu102
fastai: 2.7.9
transformers: 4.22.0


In [ ]:
# | export
def build_train_config(train_config_cls, args):
    train_config = train_config_cls()

    for arg in vars(args):
        if hasattr(train_config, arg):
            setattr(train_config, arg, getattr(args, arg))

    return train_config

In [ ]:
# | export
def run_experiment(
    experiment_name: str,
    data_path: str,
    model_output_path: str,
    log_output_path: str,
    log_preds: bool,
    log_n_preds: int,
    use_wandb: bool,
    verbose: bool,
    args=None,
):
    if experiment_name == "topic_segmentation":
        train_config = build_train_config(topic_segmentation.TopicSegmentationConfig, args)

        trainer = topic_segmentation.TopicSegmentationModelTrainer(
            experiment_name=experiment_name,
            train_config=train_config,
            data_path=data_path,
            model_output_path=model_output_path,
            log_output_path=log_output_path,
            log_preds=log_preds,
            log_n_preds=log_n_preds,
            use_wandb=use_wandb,
            verbose=verbose,
        )
    elif experiment_name == "headline_summarization":
        pass
    elif experiment_name == "content_summarization":
        pass

    # run training
    results_df, raw_df, train_df, train_val_idxs = trainer.train()

In [ ]:
# | export
def run_tuning():
    pass

In [ ]:
# | export
def add_required_args(parser):
    # define other `ModelTrainer` args
    parser.add_argument("--task", type=str, default="train", help="Options: train | tune")
    parser.add_argument("--data_path", type=str, default="data")
    parser.add_argument("--models_path", type=str, default="models")
    parser.add_argument("--logs_path", type=str, default="logs")
    parser.add_argument("--log_preds", type=bool, default=False)
    parser.add_argument("--log_n_preds", type=int, default=10)
    parser.add_argument("--use_wandb", type=bool, default=False)
    parser.add_argument("--verbose", type=bool, default=False)

    return parser

In [ ]:
# | export
if __name__ == "__main__" and not IN_NOTEBOOK:
    train_config_attrs = [(k, type(v), v) for k, v in vars(training.TrainConfig).items() if not k.startswith("_")]

    # instantiate argparser
    parser = argparse.ArgumentParser()
    subparsers = parser.add_subparsers(dest="subcommand", help="Sub-commands help")

    # instantiate argparser for topic segmentation
    parser_topic_segmentation = subparsers.add_parser("topic_segmentation", help="Topic segmentation")
    add_required_args(parser_topic_segmentation)
    train_config_attrs += [
        (k, type(v), v) for k, v in vars(topic_segmentation.TopicSegmentationConfig).items() if not k.startswith("_")
    ]
    for attr in train_config_attrs:
        parser_topic_segmentation.add_argument(f"--{attr[0]}", type=attr[1], default=attr[2])

    # # instantiate argparser for headline summarization
    # parser_headline_summarization = subparsers.add_parser("headline_summarization", help="Headling summarization")
    # add_required_args(parser_headline_summarization)
    # train_config_attrs += [(k, type(v), v) for k, v in vars(summarization.HeadlineSummarizationConfig).items() if not k.startswith("_")]
    # for attr in train_config_attrs:
    #     parser_headline_summarization.add_argument(f"--{attr[0]}", type=attr[1], default=attr[2])

    # # instantiate argparser for headline summarization
    # parser_content_summarization = subparsers.add_parser("content_summarization", help="Content summarization")
    # add_required_args(parser_content_summarization)
    # train_config_attrs += [(k, type(v), v) for k, v in vars(summarization.ContentSummarizationConfig).items() if not k.startswith("_")]
    # for attr in train_config_attrs:
    #     parser_content_summarization.add_argument(f"--{attr[0]}", type=attr[1], default=attr[2])

    # get the arg values
    args = parser.parse_args()
    experiment = args.subcommand

    # run the specific task task
    if args.task == "train":
        run_experiment(
            experiment_name=experiment,
            data_path=args.data_path,
            model_output_path=args.models_path,
            log_output_path=args.logs_path,
            log_preds=args.log_preds,
            log_n_preds=args.log_n_preds,
            use_wandb=args.use_wandb,
            verbose=args.verbose,
            args=args,
        )

    elif args.task == "tune":
        pass

## Export -

In [ ]:
# | hide
nbdev.nbdev_export()